In [12]:
# Mathematical and Data Managment
import numpy as np
import pandas as pd
import os

# Graph Managment
import graph_tool.all as gt
from utils import *

# Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

# Miscellaneous
import os
from glob import glob
from tqdm import tqdm
from datetime import datetime, timedelta
import concurrent.futures
import re

In [2]:
os.listdir('/mnt/disk2/Data/')

['Daily_graphs',
 'ACLED Colombia (2018-01-01-2023-10-31).csv',
 'RawData',
 'Proximity Between Groups',
 'Pickle',
 'deprecated',
 'Tweets_DataFrames',
 '3_Day_Graphs']

In [3]:
# Load graphs
files = glob('/mnt/disk2/Data/Daily_graphs/Graphs/*.graphml')
files = np.sort(files)

### Index of Proximity Between Groups

$$Prox_{j\rightarrow k}=\frac{W_{jk}}{(T_k/\sum_{m\in G} T_m)}$$

#### Measure $W_{jk}$

In [ ]:
results = []

for file in tqdm(files):    
    # Importamos el grafo
    g = gt.load_graph(file)
    graph_date = re.search("(\d{4}-\d{2}-\d{2})", file).group(1)

    # Número de vertices/individuos
    n_individuos = g.num_vertices()
    
    # Identifica las afiliaciones políticas únicas y asigna índices
    political_labeling = np.array([g.vp["Political Label"][j] for j in range(n_individuos)]) 
    unique_affiliations = np.unique(political_labeling)
    affiliation_to_index = {affiliation: i for i, affiliation in enumerate(unique_affiliations)}
            
    # Contamos el número de rts de cada individuo hacia cada afiliación política
    # +2 es porque necesitamos una columna de mismo y otros 
    results_matrix = np.zeros((n_individuos, len(unique_affiliations) + 2))
    for e in g.edges():
        s = int(e.source())
        t = int(e.target())
        rts = g.ep['Number of rts'][e]
        affiliation_index = affiliation_to_index[political_labeling[t]]
        results_matrix[s, affiliation_index] += rts
        # Si es un rt a alguien de la misma afiliación política
        if political_labeling[s] == political_labeling[t]:
            results_matrix[s, len(unique_affiliations)] += rts
        # Si es un rt a alguien de diferente afiliación política
        else:
            results_matrix[s, len(unique_affiliations) + 1] += rts
    
    # Calcular la matriz normalizada como un porcentaje del total de RTs salientes por nodo
    total_rts_por_nodo = results_matrix[:, 0:len(unique_affiliations)].sum(axis = 1, keepdims = True)
    total_rts_por_nodo2 = total_rts_por_nodo[:, [0]*results_matrix.shape[1]]
    # Calculamos W_jk
    with np.errstate(divide = 'ignore', invalid = 'ignore'):
        results_matrix_normalizada = np.divide(results_matrix, total_rts_por_nodo2)
    # Luego, reemplaza los valores donde total_rts_por_nodo2 es 0 con NaN
    # Esto incluye manejar divisiones 0/0 y valores/0
    results_matrix_normalizada[total_rts_por_nodo2 == 0] = np.nan

    # Construir diccionario para consolidar resultados
    temp = {
        "Nodo_ID": list(range(n_individuos)),
        "Political_Affiliation": political_labeling,
        "Date": graph_date,
        "Total_RTs": total_rts_por_nodo.flatten()
    }

    additional_categories = np.array(["Mismo", "Otros"])

    # Concatena unique_affiliations con additional_categories
    extended_affiliations = np.concatenate((unique_affiliations, additional_categories))

    # Añade las columnas de RTs por afiliación política
    for i, affiliation in enumerate(extended_affiliations):
        temp[f"rts_j_{affiliation}"] = results_matrix[:, i]
        temp[f"W_j_{affiliation}"] = results_matrix_normalizada[:, i]

    df_temp = pd.DataFrame(temp)
    results.append(df_temp)

W_jk = pd.concat(results, ignore_index = True)
W_jk.to_pickle(path = f"../../../Data/Proximity Between Groups/W_jk.gzip", compression = "gzip")

#### Denominador

In [5]:
results = []

for file in tqdm(files):  
    # Importamos el grafo
    g = gt.load_graph(file)
    graph_date = re.search("(\d{4}-\d{2}-\d{2})", file).group(1)

    # Número de vertices/individuos
    n_individuos = g.num_vertices()

    # Vamos a calcular el número de tweets por día para cada afiliación política

    # Identifica las afiliaciones políticas únicas y asigna índices
    political_labeling = np.array([g.vp["Political Label"][j] for j in range(n_individuos)]) 
    unique_affiliations = np.unique(political_labeling)
    affiliation_to_index = {affiliation: i for i, affiliation in enumerate(unique_affiliations)}
                
    # Contamos el número de tweets de cada individuo según su afiliación política
    # +2 es porque necesitamos una columna de mismo y otros 
    results_matrix = np.zeros(len(unique_affiliations))
    for v in g.vertices():
        n = g.vp["Tweets"][v]
        pl = g.vp["Political Label"][v]
        affiliation_index = affiliation_to_index[pl]
        results_matrix[affiliation_index] += n

    # Ahora calculamos el denominador para cada afiliación
    total = results_matrix.sum()
    denominadores = results_matrix/total


    # Ahora vamos a construir el denominador para cada individuo
    denominador = np.zeros((n_individuos, 2))
    for v in g.vertices():
        pl = g.vp["Political Label"][v]
        affiliation_index = affiliation_to_index[pl]
        mismo = results_matrix[affiliation_index]/total
        otros = 1 - mismo
        denominador[int(v), :] = [mismo, otros]

    # Construir diccionario para consolidar resultados
    temp = {
        "Nodo_ID": list(range(n_individuos)),
        "Political_Affiliation": political_labeling,
        "Date": graph_date,
        "Denominador Centro": denominadores[0],
        "Denominador Derecha": denominadores[1],
        "Denominador Izquierda": denominadores[2],
        "Denominador Sin Clasificar": denominadores[3],
        "Denominador Mismo": denominador[:, 0].flatten(),
        "Denominador Otros": denominador[:, 1].flatten()
    }

    df_temp = pd.DataFrame(temp)
    results.append(df_temp)
denominador = pd.concat(results, ignore_index = True)

100%|██████████| 63/63 [01:39<00:00,  1.58s/it]


In [ ]:
denominador.to_pickle(path = f"../../../Data/Proximity Between Groups/denominador.gzip", compression = "gzip")

#### Proximidad

In [6]:
W_jk.shape

(2328732, 16)

In [7]:
denominador.shape

(2328732, 9)

In [8]:
num = W_jk[["W_j_Centro", "W_j_Derecha", "W_j_Izquierda", "W_j_Sin Clasificar", "W_j_Mismo", "W_j_Otros"]].values
dem = denominador.iloc[:, 3::].values
proximidad = pd.DataFrame(num/dem, columns = ["P_Centro", "P_Derecha", "P_Izquierda", "P_Sin Clasificar", "P_Mismo", "P_Otros"])
proximidad = pd.concat([W_jk.iloc[:, :4], proximidad], axis = 1)
proximidad

,Nodo_ID,Political_Affiliation,Date,Total_RTs,P_Centro,P_Derecha,P_Izquierda,P_Sin Clasificar,P_Mismo,P_Otros
0,0,Sin Clasificar,2021-04-28,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Izquierda,2021-04-28,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Centro,2021-04-28,1.0,0.000000,0.0,0.000000,3.939678,0.000000,1.093606
3,3,Izquierda,2021-04-28,3.0,0.000000,0.0,1.384536,1.313226,1.384536,0.642891
4,4,Izquierda,2021-04-28,28.0,0.417253,0.0,1.928461,0.140703,1.928461,0.137762
...,...,...,...,...,...,...,...,...,...,...
2328727,36959,Izquierda,2021-06-29,3.0,0.000000,0.0,0.817408,1.959060,0.817408,1.125732
2328728,36960,Derecha,2021-06-29,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2328729,36961,Sin Clasificar,2021-06-29,1.0,0.000000,0.0,0.000000,2.938590,2.938590,0.000000
2328730,36962,Izquierda,2021-06-29,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
proximidad.to_pickle(path = f"../../../Data/Proximity Between Groups/proximidad.gzip", compression = "gzip")

In [16]:
import pandas as pd
import graph_tool.all as gt
proximidad = pd.read_pickle("../../../Data/Proximity Between Groups/proximidad.gzip", compression='gzip')
W_jk = pd.read_pickle(f"../../../Data/Proximity Between Groups/W_jk.gzip", compression = "gzip")
denominador = pd.read_pickle(f"../../../Data/Proximity Between Groups/denominador.gzip", compression = "gzip")

In [17]:
from utils.Proximity import individual_proximity_to_h, individual_proximity_to_others, at_random_scenario

# Valores de prueba
grupo = 'Izquierda'
vertice = 3
fecha = '2021-05-04'

G = gt.load_graph(f'/mnt/disk2/Data/Daily_graphs/Graphs/starting_{fecha}.graphml')

In [18]:
w_jk_grupo = individual_proximity_to_h(G,vertice,'Political Label',grupo)
w_jk_otros = individual_proximity_to_others(G,vertice,'Political Label')
den_grupo = at_random_scenario(G,'Political Label', grupo, 'Proximity to Group')
den_otros = at_random_scenario(G,'Political Label', grupo, 'Proximity to Others')

print('Calculos Fernando')
print()
print(f"Proximidad del Nodo {vertice} en fecha {fecha} a grupo {grupo}:")
print(f'Numerador:  {w_jk_grupo}')
print(f'Denominador:  {den_grupo}')
print(f"proximidad a {grupo}: {w_jk_grupo/den_grupo}")
print()
print(f"Proximidad del Nodo {vertice} en fecha {fecha} a otros grupos")
print(f'Numerador:  {w_jk_otros}')
print(f'Denominador:  {den_otros}')
print(f"proximidad a Otros: {w_jk_otros/den_otros}")

Calculos Fernando

Proximidad del Nodo 3 en fecha 2021-05-04 a grupo Izquierda:
Numerador:  0.8571428571428568
Denominador:  0.5876857957770661
proximidad a Izquierda: 1.4585053157691206

Proximidad del Nodo 3 en fecha 2021-05-04 a otros grupos
Numerador:  0.1428571428571428
Denominador:  0.4123142042229339
proximidad a Otros: 0.3464764041451782


In [21]:
proximidad_grupo = proximidad[f"P_{grupo}"][(proximidad["Nodo_ID"] == vertice) & (proximidad["Date"] == fecha)].iloc[0]
W_jk_grupo = W_jk[f"W_j_{grupo}"][(W_jk["Nodo_ID"] == vertice) & (W_jk["Date"] == fecha)].iloc[0]
den_grupo = denominador[f"Denominador {grupo}"][(denominador["Nodo_ID"] == vertice) & (denominador["Date"] == fecha)].iloc[0]
print('Calculos Lucas')
print()
print(f"Proximidad del Nodo {vertice} en fecha {fecha} a grupo {grupo} En DataFrame")
print(f"Numerador: {W_jk_grupo}")
print(f"Denominador: {den_grupo}")
print(f"Proximidad a {grupo}: {proximidad_grupo}")
print()

W_jk_otros = W_jk[f"W_j_Otros"][(W_jk["Nodo_ID"] == vertice) & (W_jk["Date"] == fecha)].iloc[0]
den_otros = denominador[f"Denominador Otros"][(denominador["Nodo_ID"] == vertice) & (denominador["Date"] == fecha)].iloc[0]
proximidad_otros = proximidad[f"P_Otros"][(proximidad["Nodo_ID"] == vertice) & (proximidad["Date"] == fecha)].iloc[0]

print(f"Proximidad del Nodo {vertice} en fecha {fecha} a otros grupos En DataFrame")
print(f"Numerador: {W_jk_otros}")
print(f"Denominador: {den_otros}")
print(f"Proximidad a Otros: {proximidad_otros}")
print(f"Numerador/Denominador: {W_jk_otros/den_otros}")

Calculos Lucas

Proximidad del Nodo 3 en fecha 2021-05-04 a grupo Izquierda En DataFrame
Numerador: 0.8571428571428571
Denominador: 0.5876857957770661
Proximidad a Izquierda: 1.4585053157691212

Proximidad del Nodo 3 en fecha 2021-05-04 a otros grupos En DataFrame
Numerador: 0.14285714285714285
Denominador: 0.41231420422293386
Proximidad a Otros: 0.3464764041451784
Numerador/Denominador: 0.3464764041451784
